# 第 4 章: Apache Spark - 2

本ノートブックでは、「基本的な操作を試してみる」節で紹介されている例を実行できます。

## 基本的な操作を試してみる

In [ ]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
CATALOG = "my_catalog"
CATALOG_URL = "http://server:8181/"
S3_ENDPOINT = "http://minio:9000"
SPARK_VERSION = pyspark.__version__
SPARK_MINOR_VERSION = '.'.join(SPARK_VERSION.split('.')[:2])
ICEBERG_VERSION = "1.8.1"

### 1. SparkSession オブジェクトを初期化する

In [ ]:
spark = (
    SparkSession.builder
        .config("spark.jars.packages", f"org.apache.iceberg:iceberg-spark-runtime-{SPARK_MINOR_VERSION}_2.12:{ICEBERG_VERSION},org.apache.iceberg:iceberg-aws-bundle:{ICEBERG_VERSION}")
        .config(f"spark.sql.catalog.{CATALOG}", "org.apache.iceberg.spark.SparkCatalog")
        .config(f"spark.sql.catalog.{CATALOG}.type", "rest")
        .config(f"spark.sql.catalog.{CATALOG}.uri", CATALOG_URL)
        .config(f"spark.sql.catalog.{CATALOG}.s3.endpoint", S3_ENDPOINT)
        .config(f"spark.sql.catalog.{CATALOG}.view-endpoints-supported", "true")
        .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
        .getOrCreate()
)

In [ ]:
%sql spark

#### (Optional) データベースの作成
データベースを作成していない場合、以下のセルを実行してください。既にデータベースが存在する場合は、本ステップにつきましてはスキップしてください。

In [ ]:
%%sql
CREATE DATABASE IF NOT EXISTS db

### 2. Iceberg テーブルを作成する

In [ ]:
%%sql
CREATE TABLE my_catalog.db.sales_iceberg_example (
    product_name string,
    price decimal(10, 2),
    customer_id bigint,
    order_id string,
    datetime timestamp,
    category string
) USING iceberg

### 3. Iceberg テーブルにデータを書き込む

In [ ]:
%%sql
INSERT INTO my_catalog.db.sales_iceberg_example VALUES
    ('tomato juice', 2.00, 1698, 'DRE8DLTFNX0MLCE8DLTFNX0MLC', TIMESTAMP '2023-07-18T02:20:58Z', 'drink'),
    ('cocoa', 2.00, 1652, 'DR1UNFHET81UNFHET8', TIMESTAMP '2024-08-26T11:36:48Z', 'drink'),
    ('espresso', 2.00, 1037, 'DRBFZUJWPZ9SRABFZUJWPZ9SRA', TIMESTAMP '2024-04-19T12:17:22Z', 'drink'),
    ('broccoli', 1.00, 3092, 'GRK0L8ZQK0L8ZQ', TIMESTAMP '2023-03-22T18:48:04Z', 'grocery'),
    ('nutmeg', 1.00, 3512, 'GR15U0LKA15U0LKA', TIMESTAMP '2024-02-27T15:13:31Z', 'grocery')

### 4. 書き込んだデータを読む

In [ ]:
%%sql
SELECT * FROM my_catalog.db.sales_iceberg_example